In [10]:
pip install --upgrade mediapipe


Note: you may need to restart the kernel to use updated packages.


In [14]:

pip install mediapipe


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to extract hand landmarks from an image
def extract_hand_landmarks(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        result = hands.process(img_rgb)
        if result.multi_hand_landmarks:
            landmarks = result.multi_hand_landmarks[0].landmark
            # Convert landmarks to a flattened array
            return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()
        else:
            return None  # Return None if no hand is detected

# Load dataset
def load_data(data_dir):
    features = []
    labels = []
    for label in ['0', '1', '2']:  # Folders representing Start, Pause, Stop
        folder_path = os.path.join(data_dir, label)
        for file in os.listdir(folder_path):
            if file.endswith('.jpg'):
                image_path = os.path.join(folder_path, file)
                landmarks = extract_hand_landmarks(image_path)
                if landmarks is not None:  # Only add if hand landmarks were detected
                    features.append(landmarks)
                    labels.append(int(label))
    return np.array(features), np.array(labels)

# Load the image dataset and extract hand landmarks
data_dir = 'data'  # Path to the dataset with folders '0', '1', '2'
X, y = load_data(data_dir)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier using hand landmarks
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the trained model
joblib.dump(clf, 'hand_landmark_random_forest_model.pkl')
print("Model trained and saved as 'hand_landmark_random_forest_model.pkl'")


C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Model trained and saved as 'hand_landmark_random_forest_model.pkl'


In [1]:
 streamlit run C:\Users\Owen Brown\anaconda3\Lib\site-packages\ipykernel_launcher.py

SyntaxError: invalid syntax (2179123029.py, line 1)

In [8]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
from collections import deque
from datetime import datetime
import time

# Load the Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Logging predictions in a list
logged_predictions = []

# Class mapping
class_map = {0: 'Start', 1: 'Pause', 2: 'Stop'}

# Variables to track consistent detection
current_class = None
class_start_time = None
consistency_duration = 10  # seconds to wait before displaying the new class
last_predictions = deque(maxlen=3)

# Streamlit app layout
st.title("Hand Pose Detection")
st.image("img_pldrn.jpg", use_column_width=True)  # Background image

# Define the video capture window layout and buttons
start_button = st.button("Start")
stop_button = st.button("Stop")
print_button = st.button("Print Results")

# Video capture placeholder
video_placeholder = st.empty()

# Initialize variables
cap = None
running = False

# Start video capture when Start button is clicked
if start_button:
    running = True
    cap = cv2.VideoCapture(0)

# Stop video capture when Stop button is clicked
if stop_button and cap:
    cap.release()
    running = False

# Capture and display video in real-time
if running and cap is not None:
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB for Mediapipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect hands and get hand landmarks
            result = hands.process(rgb_frame)

            detected_class = None

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    # Draw hand landmarks on the frame
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Predict hand pose based on landmarks
                    predicted_class = predict_hand_pose(hand_landmarks.landmark)
                    detected_class = class_map[predicted_class]

                    # Get bounding box coordinates around the hand
                    h, w, _ = frame.shape
                    x_min, x_max, y_min, y_max = w, 0, h, 0
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * w), int(lm.y * h)
                        x_min, x_max = min(x, x_min), max(x, x_max)
                        y_min, y_max = min(y, y_min), max(y, y_max)

                    # Draw bounding box around the hand
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Check if the detected class is consistent for 10 seconds
            if detected_class:
                if detected_class == current_class:
                    elapsed_time = time.time() - class_start_time
                    if elapsed_time >= consistency_duration:
                        if len(last_predictions) == 0 or last_predictions[-1] != current_class:
                            last_predictions.append(current_class)
                            logged_predictions.append({
                                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                "prediction": current_class
                            })
                else:
                    current_class = detected_class
                    class_start_time = time.time()

            # Display the frame with landmarks and bounding box
            video_placeholder.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), channels="RGB")

# Print results to a spreadsheet when Print button is clicked
if print_button and logged_predictions:
    df = pd.DataFrame(logged_predictions)
    file_name = f'logged_predictions_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
    df.to_csv(file_name, index=False)
    st.success(f"Results saved as {file_name}")
    st.dataframe(df)


RuntimeError: Runtime hasn't been created!

In [3]:
pip install streamlit opencv-python mediapipe joblib pandas


Note: you may need to restart the kernel to use updated packages.
